<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/TFTimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_datareader

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from datetime import date
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.contrib.learn import ModeKeys
import tensorflow.contrib.rnn as rnn
from datetime import datetime, date, timedelta





In [0]:

from pandas.tseries.offsets import BDay
TIMESERIES_COL = 'rawdata'
N_OUTPUTS = 5 # in each sequence, 1-14 are features, and 14-20 is label 
SEQ_LEN = 20
DEFAULTS = 0.0
LSTM_SIZE = 4  # number of hidden layers in each of the LSTM cells
N_INPUTS = SEQ_LEN - N_OUTPUTS
BATCH_SIZE = 20
ROOT_DIR = '/home/mmistroni/tf_logs/rnn-run-{}'


def get_prices(startdate, enddate):
  import pandas_datareader as pdr
  from datetime import date, timedelta
  print ('--Start:{%s}, end:{%s}', startdate, enddate)
  
  stock_data = pdr.get_data_yahoo('XOM', startdate, enddate)
  adjClose = np.stack(stock_data['Close'])
  return adjClose
  
def create_training_data2(inputData):
  print ('Len of input dat ais {}', len(inputData))
  return [np.array(inputData[i * SEQ_LEN: (i + 1) * SEQ_LEN]) 
       for i in range(len(inputData) // SEQ_LEN)]

def create_training_data(inputData):
  print ('AdjClose is of shape {}', inputData.shape)
  return inputData.T.reshape(-1,SEQ_LEN)
  
  
def create_time_series(numDays=200):
  endDate = date.today() - BDay(15) # training from 20 days ago
  startDate = endDate - BDay(220)
  prices =  get_prices(startDate, endDate)
  return create_training_data2(prices)
  


In [32]:
all_timeseries =  create_time_series() #[create_time_series() for i in range(0, SEQ_LEN * 4)]
all_data = np.stack(all_timeseries)
print('All data shape is{0} and type {1}'.format(all_data.shape,type(all_data)))
X, y = all_data[...,0:-N_OUTPUTS], all_data[...,-N_OUTPUTS:]
print ('X is fo type {0}, y  of type {1}'.format(type(X[0][0]), type(y)))
print ('X.shape is {0}, y shap is {1}'.format(X.shape, y.shape))
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.1,
                                                    random_state=1)



--Start:{%s}, end:{%s} 2018-06-01 00:00:00 2019-04-05 00:00:00
Len of input dat ais {} 213
All data shape is(10, 20) and type <class 'numpy.ndarray'>
X is fo type <class 'numpy.float64'>, y  of type <class 'numpy.ndarray'>
X.shape is (10, 15), y shap is (10, 5)


<h3> Creating RNN Model </h3>


In [0]:

# create the inference model
def simple_rnn(features, labels, mode, params):
  # 0. Reformat input shape to become a sequence
  print ('IN Features are:{0}'.format(features))
  x = tf.split(features[TIMESERIES_COL], N_INPUTS, 1)
  #print 'x={}'.format(x)
    
  # 1. configure the RNN
  lstm_cell = rnn.BasicLSTMCell(LSTM_SIZE, forget_bias=1.0)
  outputs, _ = tf.nn.static_rnn(lstm_cell, x, dtype=tf.float32)

  # slice to keep only the last cell of the RNN
  outputs = outputs[-1]
  #print 'last outputs={}'.format(outputs)
  
  # output is result of linear activation of last layer of RNN
  weight = tf.Variable(tf.random_normal([LSTM_SIZE, N_OUTPUTS]))
  bias = tf.Variable(tf.random_normal([N_OUTPUTS]))
  predictions = tf.matmul(outputs, weight) + bias
    
  # 2. loss function, training/eval ops
  if mode == ModeKeys.TRAIN or mode == ModeKeys.EVAL:
     loss = tf.losses.mean_squared_error(labels, predictions)
     train_op = tf.contrib.layers.optimize_loss(
         loss=loss,
         global_step=tf.train.get_global_step(),
         learning_rate=0.01,
         optimizer="SGD")
     eval_metric_ops = {
      "rmse": tf.metrics.root_mean_squared_error(labels, predictions)
     }
  else:
     loss = None
     train_op = None
     eval_metric_ops = None
  
  # 3. Create predictions
  predictions_dict = {"predicted": predictions}

  # 4. Create export outputs  
  export_outputs = {"predicted": tf.estimator.export.PredictOutput(predictions)}

  # 5. return ModelFnOps
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=predictions_dict,
      loss=loss,
      train_op=train_op,
      eval_metric_ops=eval_metric_ops,
      export_outputs=export_outputs)








<h3> Creating Serving Function , Train Function and Test Function </h3>

In [0]:
def serving_input_receiver_fn():
  feature_placeholders = {
    TIMESERIES_COL: tf.placeholder(tf.float32, [None, N_INPUTS])
  }

  features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
  }

  features[TIMESERIES_COL] = tf.squeeze(features[TIMESERIES_COL], axis=[2], name='timeseries')
  
  print('serving: features={}'.format(features[TIMESERIES_COL]))

  return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)



# Creating a TrainFn and a TestFn
def _train_fn(X, y, batch_size):
    
    def _train():
        """An input function for training"""
        # Convert the inputs to a Dataset.
        # TODO need to be refactored according to https://medium.com/google-cloud/how-to-do-time-series-prediction-using-rnns-and-tensorflow-and-cloud-ml-engine-2ad2eeb189e8
        # this is not good.
        X_32 = tf.cast(X, tf.float32)
        y_32 = tf.cast(y, tf.float32)
        inputs = tf.concat(X_32, axis=1)
        label = tf.concat(y_32, axis=1)
        features, labels = {TIMESERIES_COL: inputs}, label
        
        return features, labels
        dataset = tf.data.Dataset.from_tensor_slices(features, labels)
        # Shuffle, repeat, and batch the examples.
        dataset = dataset.repeat(None).batch(batch_size)
        # This will now return batches of features, label
        return dataset.make_one_shot_iterator().get_next()
    return _train

def _test_fn(X, y, batch_size):
    def _test():
        """An input function for training"""
        # Convert the inputs to a Dataset.
        X_32 = tf.cast(X, tf.float32)
        y_32 = tf.cast(y, tf.float32)
        inputs = tf.concat(X_32, axis=1)
        label = tf.concat(y_32, axis=1)
        
        features, labels = {TIMESERIES_COL: inputs}, label
        
        return features, labels
        
        dataset = tf.data.Dataset.from_tensor_slices(features, labels)
        # Shuffle, repeat, and batch the examples.
        dataset = dataset.repeat(1).batch(batch_size)
        # This will now return batches of features, label
        return dataset.make_one_shot_iterator().get_next()
    return _test  

 


<h3> Creating Experiment Function and running model </h3>

In [35]:
def experiment_fn(output_dir, X, y, x_tst, y_tst):
    # run experiment
    train_spec = tf.estimator.TrainSpec(
          input_fn=_train_fn(X, y, BATCH_SIZE), max_steps=1500)
    exporter = tf.estimator.FinalExporter('timeseries',
    serving_input_receiver_fn)
    eval_spec = tf.estimator.EvalSpec(
            input_fn=_test_fn(x_tst, y_tst, BATCH_SIZE),
            exporters=[exporter])
    
    estimator = tf.estimator.Estimator(model_fn=simple_rnn, model_dir=output_dir)

    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    return estimator

tf.logging.set_verbosity(tf.logging.INFO)    
    
output_dir = ROOT_DIR.format(datetime.utcnow().strftime('%Y%m%d%H%M%S'))  
  
estimator = experiment_fn(output_dir, X_train, y_train, X_test, y_test)


print('Xtrain is:{} and of shape:{}', type(X_train), X_train.shape)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/mmistroni/tf_logs/rnn-run-20190426203340', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbea3f32e80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-

<h3> Now running some predictions with last 15 days </h3>

In [46]:
from datetime import date
enddt  = date.today()
startdt = enddt - BDay(N_INPUTS)

tf.logging.set_verbosity(tf.logging.INFO)    

def _predict_fn(X) :
    def _predict():
        """An input function for training"""
        # Convert the inputs to a Dataset.
        X_32 = tf.cast(X, tf.float32)
        inputs = tf.concat(X_32, axis=1)
        features = {TIMESERIES_COL: inputs}
        return features
    return _predict

def get_prices2(startdate=None, enddate=None):
  import pandas_datareader as pdr
  from datetime import date, timedelta
  print ('--Start:{%s}, end:{%s}', startdate, enddate)
  
  if not startdate and not enddate:
    last_check = date.today() - BDay(SEQ_LEN * 2)
    enddate = last_check
    startdate = last_check - BDay(122)#timedelta(days=120)
  stock_data = pdr.get_data_yahoo('XOM', startdate, enddate)
  adjClose = np.stack(stock_data['Close'])
  return adjClose


prices = get_prices2(startdt, enddt).reshape(-1,N_INPUTS)
print ('Prices is of shape:{}', prices.shape)


print ('-------- PREDICTING -------')
print (type(prices))


def _predict_fn(X):
    def _predict():
        """An input function for training"""
        # Convert the inputs to a Dataset.
        X_32 = tf.cast(X, tf.float32)
        inputs = tf.concat(X_32, axis=1)
        features = {TIMESERIES_COL: inputs}
        return features
        
    return _predict 


acc = []
acc += prices[0].tolist()

print ("Before ranges X_train shape is:{}".format(X_train.shape))

for i in range(0,5):
  # Given last 15 days, predict the next 5
  candidate = np.array(acc[-15:]).reshape(-1,15)  # this is the Train #
  pred = estimator.predict(input_fn = _predict_fn(candidate))
  item = next(pred)
  vals = item['predicted'] # this is the predicted
  # Next 5 predicted. Now,  we need to retrain the model to add the 5 predicted
  # Then, as test data, pick the previous 15 and then predict the next one
  
  
  print ('Round {} Prediction: on {}={}'.format(i, candidate, vals))
  print ('{}={}'.format(type(vals), vals.tolist()))
  print("====== Now adding prediction to current train....")
  print("Xtrain of type:{}, candidate of type:{}".format(type(X_train), type(candidate)))
  print("SHapes.Train:{}, Candidate:{}".format(X_train.shape, candidate.shape))
  
  #X_train = np.vstack([X_train , candidate])
  #y_train = np.vstack([y_train , vals])
  
  # So Now we have to 
  # 1. retrain the model
  # 2. get the most recent 15 days to 
  acc += vals.tolist()

print('-------------- END OF STORY')
from pprint import pprint
future_days = len(acc) - 15
pprint('Accumlator has length:{}'.format(len(acc)))
pprint('We have prediction for the next:{} days'.format(future_days))
pprint(acc)
print("calculating business days to zip ")
# we got start date, todays - 15
# end date is  today + 
dts = [startdt + BDay(offset) for offset in range(0, len(acc))]
       
print('Start Date:{}={}'.format(dts[0], startdt))
print('End date:{}'.format(dts[-1]))
       
zipped = [tpl for tpl in zip(acc, dts)]
       
       
print('===== FULL PREDICTIONS======')
pprint(zipped)
       
       
       
       
       








--Start:{%s}, end:{%s} 2019-04-05 00:00:00 2019-04-26
Prices is of shape:{} (1, 15)
-------- PREDICTING -------
<class 'numpy.ndarray'>
Before ranges X_train shape is:(9, 15)
INFO:tensorflow:Calling model_fn.
IN Features are:{'rawdata': <tf.Tensor 'concat:0' shape=(1, 15) dtype=float32>}
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/mmistroni/tf_logs/rnn-run-20190426203340/model.ckpt-1500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Round 0 Prediction: on [[82.48999786 83.         81.93000031 81.55999756 81.94999695 80.91999817
  80.65000153 81.19999695 81.43000031 81.12999725 82.90000153 83.37999725
  81.76000214 82.22000122 80.48999786]]=[78.73065  78.7193   78.82818  79.246025 79.111046]
<class 'numpy.ndarray'>=[78.73065185546875, 78.71929931640625, 78.82817840576172, 79.24602508544922, 79.11104583740234]
====== Now adding prediction to current train....
Xtrain of type

Installing libraries and downloading data as DataFrame
